In [7]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
import sys
sys.path.append(str("./dr_sasa_python/build/"))
sys.path.append(str("./dr_sasa_python/dr_sasa_python/bindings/python"))
import dr_sasa_python as sasa
from dr_sasa_python.utils.structure_parser import StructureData, parse_pdb_file, superimpose_structures
from dr_sasa_python.utils.utils import convert_to_dataframes


In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/data/PRODIGYdataset/1ACB.pdb"
calc = sasa.SimpleSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), print_output=True)
dfs = convert_to_dataframes(result)
dfs.keys()
# Direct atom access (dictionary)
np.asarray([result["atoms"][str(i)]["surface"]["sasa"] for i in list(result["atoms"].keys())]).sum()

In [ ]:
#Via dataframe
atoms

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/data/PRODIGYdataset/1ACB.pdb"
calc = sasa.DecoupledSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), chains=[["A"], ["B"]],  print_output=True)
dfs = convert_to_dataframes(result)
dfs.keys()

In [ ]:
# Easy dataframes access
overlaps = dfs["overlaps"]
atoms = dfs["atoms"] # not actual results value accessible for DecoupledSASA check the overlaps dataframe and contact dataframe
residues = dfs["residues"]
contacts = dfs["contacts"]
#Interface A/B (A^2):	766.656 
print(overlaps.area.sum()/2)
print(overlaps.buried_area.sum()/2)
print((result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum() + result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum())/2)
print(result["intra_bsa_matrix"]["atom_matrix"].sum() /2)
print(overlaps.normalized_area.sum())  # almost

In [ ]:
test_pdb = "/home/alessio/dr_sasa_python/tests/data/3i40.pdb"
calc = sasa.GenericSASA(probe_radius=1.4)
result = calc.calculate(str(test_pdb), chains=[["A"], ["B"]], include_matrix=True, print_output=True)
dfs = convert_to_dataframes(result)

In [ ]:
overlaps = dfs["overlaps"]
atoms = dfs["atoms"] # not actual results value accessible for DecoupledSASA check the overlaps dataframe and contact dataframe
residues = dfs["residues"]
contacts = dfs["contacts"]
result["intra_bsa_matrix"]["residue_matrix"][:len(residues[residues.chain =="A"].index)].sum() # same as #A <--- B buried surface (A^2):	791.494
result["inter_bsa_matrix"]["A_vs_B"]["matrix"].sum()
##
result["intra_bsa_matrix"]["residue_matrix"][len(residues[residues.chain =="A"].index):].sum() # same as A ---> B buried surface (A^2):	741.817
result["inter_bsa_matrix"]["B_vs_A"]["matrix"].sum()
##
overlaps.area.sum()/2 #Interface A/B (A^2):	766.656
result["intra_bsa_matrix"]["atom_matrix"].sum()/2
##
atoms[atoms.chain == "A"].sasa.sum() #Object A complexed surface (A^2):	1198.94 calculation based on atom contact surfaces
residues[residues.chain == "A"].dsasa.sum() # close values (calculated based on standards NACESS sasa values of per residues)
atoms[atoms.chain == "B"].sasa.sum() #Object B complexed surface (A^2):	2165.03
## 
atoms.sasa.sum() #Selected complex surface (A^2):	3363.97
# Object A uncomplexed surface (A^2):	1990.44
residues[residues.chain == "A"].dsasa.sum()


In [ ]:
residues[residues.chain == "A"].dsasa.sum() - residues[residues.chain == "A"].total_sasa.sum()
residues[residues.chain == "B"].dsasa.sum() - residues[residues.chain == "B"].total_sasa.sum()